In [95]:
import json,urllib.request
import dload
import pandas as pd

import networkx as nx
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import warnings
import mpld3

from scipy.optimize import curve_fit


def exp_fit_plot(y):
    N=len(y)
    x = np.linspace(0, 1, num = N)     # Column values as a function of depth
    

    plt.plot(x, y, 'ro',label="Original Data")

    def func(x, I0, k):     # a = I0, b = k
        return I0 * np.exp(k*x)    

    popt, pcov = curve_fit(func, x, y,method='lm')

    print ("I(0) = %s , k = %s" % (popt[0], popt[1]))

    plt.plot(x, func(x, *popt), label="Fitted Curve")
    return popt[1]

def log_fit_plot(y):
    N=len(y)
    x = np.linspace(0, 1, num = N)     # Column values as a function of depth
    

    plt.plot(x, y, 'ro',label="Original Data")
    N0=y[0]
    def func(x, N0, k,r):     # a = I0, b = k
        return (k*N0)/(N0+(k-N0)*np.exp(-r*x))     

    popt, pcov = curve_fit(func, x, y,method='lm')

    print ("N0=%s, k = %s , r = %s" % (popt[0], popt[1],popt[2]))

    plt.plot(x, func(x, *popt), label="Fitted Curve")
    #return popt

def exp_fit(y):
    N=len(y)
    x = np.linspace(0, 1, num = N)     # Column values as a function of depth


    def func(x, I0, k):     # a = I0, b = k
        return I0 * np.exp(k*x)    

    popt, pcov = curve_fit(func, x, y)

    return popt[1]


def log_fit(y):
    N=len(y)
    x = np.linspace(0, 1, num = N)     # Column values as a function of depth
    
    N0=y[0]
    def func(x, N0, k,r):     # a = I0, b = k
        return (k*N0)/(N0+(k-N0)*np.exp(-r*x))     
    try:
        popt, pcov = curve_fit(func, x, y,method='lm')
    except:
        popt=[0.0, 0.0, 0.0]
        pass
    
    return popt

def log_fun(x, N0, k,r):     # a = I0, b = k
        return (k*N0)/(N0+(k-N0)*np.exp(-r*x)) 

## Analisi dati Nazionali

In [149]:
url='https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-andamento-nazionale.json'
j = dload.json(url)
df=pd.DataFrame.from_dict(j)

#"ricoverati_con_sintomi"
#"terapia_intensiva"
#"totale_ospedalizzati"
#"isolamento_domiciliare"
#"totale_attualmente_positivi"
#"nuovi_attualmente_positivi"
#"dimessi_guariti"
#"deceduti"
#"totale_casi"
#"tamponi"


VAR='totale_casi'
#df.set_index(pd.to_datetime(df.data), drop=True).plot(legend=False, logy=True, figsize=(15,10))
y=df[VAR]
log_fit_plot(y)


## Analisi dati provinciali

In [139]:
url='https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-province.json'
j = dload.json(url)
df=pd.DataFrame.from_dict(j)
df = df[df.denominazione_provincia != 'In fase di definizione/aggiornamento'] #elimino i dati in corso di validazione

date=df.data.unique()
province=df.denominazione_provincia.unique()

#variabile che si vuole visualizzare
VAR='totale_casi'

#Metto le province sulle colonne
casi_p=pd.DataFrame(date, columns=['data'])
for p in province:
    df_p=df[df.denominazione_provincia == p]
    casi=pd.DataFrame(list(df_p[VAR]),columns=[p])
    casi_p= pd.concat([casi_p, casi], axis=1)

In [176]:
##Plotto tutto in loglog
#https://pandas.pydata.org/pandas-docs/version/0.22.0/generated/pandas.DataFrame.plot.html
#casi_p.set_index(pd.to_datetime(casi_p.data), drop=True).plot(legend=False, logy=True, figsize=(15,10))
    

### Fit di una provincia specifica

In [150]:
log_fit_plot(casi_p['Alessandria'])

## Fit del modello logistico sui dati provinciali

In [158]:
K=[]
for p in province:
    y=casi_p[p]
    y=y[0:-1]
    fit=log_fit(y)
    #N0->fit[0]; k=fit[1] r= fit[2]
    K.append(fit[2])
K=pd.DataFrame(K, columns=['K_exp'])
P=pd.DataFrame(province, columns=['province'])
exp_data=pd.merge(P,K, left_index=True, right_index=True)

In [151]:
#Istogramma dei K
K.hist(bins=15)

### Mappa per province del valore selezionato del modello logistico

In [ ]:
fig = px.choropleth(exp_data, geojson=geo_prov, locations='province', color='K_exp',
                           color_continuous_scale="Reds",
                           #range_color=(0, 12),
                           featureidkey='properties.NOME_PRO',
                           projection='mercator',
                           labels={'numero_casi':'Numero di Casi'}             
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Mappa dei casi totali per provincia

In [145]:
import plotly.express as px
import geojson
import json

#Importo i profili delle province
with open('province.geojson',encoding='utf-8', errors='ignore') as json_data:
     geo_prov = json.load(json_data, strict=False)

VAR='numero_casi'
N=len(casi_p_t.columns)-1
casi_p_t=casi_p.transpose()
df2=pd.DataFrame(casi_p_t[N])
df2=df2.drop('data', axis=0)
df2.columns=[VAR]
df2=df2.astype({VAR: 'int64'})


In [ ]:
fig = px.choropleth(df2, geojson=geo_prov, locations=df2.index, color=VAR,
                           color_continuous_scale="Reds",
                           #range_color=(0, 12),
                           featureidkey='properties.NOME_PRO',
                           projection='mercator',
                           labels={'numero_casi':'Numero di Casi'}
                    
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Mappa delle variazioni percentuali per provincia

In [153]:
df3=pd.DataFrame([casi_p_t[N-1],casi_p_t[N]])
df3=df3.drop('data', axis=1)
df3.iloc[0].apply(int)
df3.iloc[1].apply(int)
df3.loc['delta']=100.0*((df3.iloc[1]-df3.iloc[0])/df3.iloc[0])
df3=df3.T

fig = px.choropleth(df3, geojson=geo_prov, locations=df3.index, color='delta',
                           color_continuous_scale="Reds",
                           #range_color=(0, 12),
                           featureidkey='properties.NOME_PRO',
                           projection='mercator',
                           labels={'numero_casi':'Numero di Casi'}
                    
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df3['delta'].hist(bins=15)

## Analisi Regionale

In [142]:
import plotly.express as px
import geojson
import json## Analisi dati regionali


url='https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-regioni.json'
j = dload.json(url)
df_r=pd.DataFrame.from_dict(j)

date=df_r.data.unique()
regioni=df_r.denominazione_regione.unique()

casi_r=pd.DataFrame(date, columns=['data'])

#"ricoverati_con_sintomi"
#"terapia_intensiva"
#"totale_ospedalizzati"
#"isolamento_domiciliare"
#"totale_attualmente_positivi"
#"nuovi_attualmente_positivi"
#"dimessi_guariti"
#"deceduti"
#"totale_casi"
#"tamponi"

for r in regioni:
    df_temp=df_r[df_r.denominazione_regione == r]
    casi=pd.DataFrame(list(df_temp[VAR]),columns=[r])
    casi_r= pd.concat([casi_r, casi], axis=1)


#I dati sul Trentino sono separati in due province
#Questo non torna con lo shapefile. Unisco le due regioni
sum_column = casi_r["P.A. Bolzano"] + casi_r["P.A. Trento"]
casi_r["Trentino-Alto Adige"] = sum_column  

VAR='deceduti'
   
#Importo i profili delle province
with open('regioni.geojson',encoding='utf-8', errors='ignore') as json_data:
     geo_reg = json.load(json_data, strict=False)

L=len(casi_r)-1
casi_r_t=casi_r.transpose()
dfR=pd.DataFrame(casi_r_t[L])
dfR=dfR[dfR.index !='data']
dfR.columns=[VAR]
dfR=dfR.astype({VAR: 'int64'})


In [154]:
import plotly.express as px

#reg= pd.read_excel('prova_regioni.xlsx', sheet_name=0)

fig = px.choropleth(dfR, geojson=geo_reg, locations=dfR.index, color=VAR,
                           color_continuous_scale="Reds",
                           featureidkey='properties.NOME_REG',
                           projection='mercator',
                           scope='europe'
                           #labels={'numero_casi':'Numero di Casi'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


## Mappa con i fit logistici delle regioni

In [246]:
K=[]
regioni=casi_r.columns
regioni.drop('data')
for r in regioni:
    y=casi_r[r]
    y=y[0:-1]
    fit=log_fit(y)
    #N0->fit[0]; k=fit[1] r= fit[2]
    K.append(fit[1])
K=pd.DataFrame(K, columns=['K_exp'])
P=pd.DataFrame(regioni, columns=['regione'])
exp_data=pd.merge(P,K, left_index=True, right_index=True)


In [ ]:

fig = px.choropleth(exp_data, geojson=geo_reg, locations=exp_data['regione'], color='K_exp',
                           color_continuous_scale="Reds",
                           featureidkey='properties.NOME_REG',
                           projection='mercator',
                           scope='europe'
                           #labels={'numero_casi':'Numero di Casi'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()